<a href="https://colab.research.google.com/github/kanzeparov/Analytics/blob/master/swopFiCalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HOW TO

1) **←** слева есть оглавление для удобной навигации


![](https://drive.google.com/uc?export=view&id=1IqXsxIjITqrkasxSjOIDAbg_aVtCG9Am)

2) В каждом блоке есть входные параметры, которые можно и нужно менять
![](https://drive.google.com/uc?export=view&id=1ojUvqrPFLUjL69P-qExQazZiWaHiiJ6D)

3) Этот калькуятор содержит технические блоки с кодом, у каждого блока они свои и скрыты для удобства. Если раскроете блок с кодом и захотите его убрать, нажмите на появившуюся белую область справа

 Выглядят блоки с кодом вот так
![](https://drive.google.com/uc?export=view&id=1xN4Atojaz_Iu2DxaS-ZN2j7loSLB9v8D)
**↓** **↓** **↓**Для дальшнейшей работы как раз необходимо выполнить ячейку снизу (выделить и нажать shift+enter)**↓** **↓** **↓**



In [ ]:
#@title

from IPython.display import HTML, display

import numpy as np
import plotly.graph_objects as go
import math
#высчитваем инвариант
def invV1(x_i,y_i,alpha):
  skeweness = 1/2*(y_i/x_i+x_i/y_i)
  return (x_i+y_i)*(skeweness)**(-alpha) + (x_i*y_i)*(skeweness)**(alpha)

def invV2_cont(x_i,y_i,alpha,betta):
  skeweness = 1/2*(y_i/x_i+x_i/y_i)
  return (x_i+y_i)*(skeweness)**(-alpha) + 2*(x_i*y_i)**(1/2)*(skeweness - betta)**((alpha))


def invV2(x_i,y_i,alpha,betta):
  scale8 = 100000000
  scale12 = 1000000000000
  skeweness = math.floor(1/2*(int(y_i*scale12/x_i)+int(x_i*scale12/y_i))/10000)
  first_sum = math.floor((x_i+y_i)*scale8/math.ceil((skeweness/scale8)**(alpha)*scale8))
  second_sum_first_part = math.floor(round(x_i*y_i/scale8)**0.5*(scale8/10000))
  second_sum_second_part = math.floor(((skeweness-betta*scale8)/scale8)**alpha*scale8)
  second_sum = 2*math.floor((second_sum_first_part*second_sum_second_part)/scale8)
  return first_sum + second_sum


# зная о монотонности функции,воспользуемся методом половинного деления
def find_how_many_you_getV2(x_balance,y_balance,x_balance_new,alpha,betta):
  actual_invarian = invV2(x_balance,y_balance,alpha,betta)
  y_left = 1
  y_right = 100*y_balance 
  for _ in range(50):
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV2(x_balance_new,y_left,alpha,betta)
    invariant_delta_in_right = actual_invarian - invV2(x_balance_new,y_right,alpha,betta)
    invariant_delta_in_mean = actual_invarian - invV2(x_balance_new,mean,alpha,betta)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance -  mean
  return y_balance -  mean
  
def find_how_many_you_getV2_cont(x_balance,y_balance,x_balance_new,alpha,betta):
  actual_invarian = invV2_cont(x_balance,y_balance,alpha,betta)
  y_left = 1
  y_right = 100*y_balance 
  for _ in range(50):
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV2_cont(x_balance_new,y_left,alpha,betta)
    invariant_delta_in_right = actual_invarian - invV2_cont(x_balance_new,y_right,alpha,betta)
    invariant_delta_in_mean = actual_invarian - invV2_cont(x_balance_new,mean,alpha,betta)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance -  mean
  return y_balance -  mean

def find_how_many_you_getV1(x_i,alpha):
  actual_invarian = invV1(x_balance,y_balance,alpha)
  y_left = 0.000000001*y_balance
  y_right = 10*y_balance 
  for _ in range(1000):
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV1(x_i,y_left,alpha)
    invariant_delta_in_right = actual_invarian - invV1(x_i,y_right,alpha)
    invariant_delta_in_mean = actual_invarian - invV1(x_i,mean,alpha)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance - y_right  
  return y_balance - y_right  

def find_how_many_you_get_uniswap(x_i):
  return y_balance - x_balance*y_balance/(x_i)

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

In [ ]:
invV2()


---


### Лучшие параметры для flat кривых

Для нахождения параметров, соответствующих наиболее "плоскому" поведению bonding curves, можно выполнить участок кода снизу.

Работает **долго**, т.к. подбор осуществляется перебором

In [ ]:
x_balance = 500000000000 #500k with 6 digits
y_balance = x_balance

In [ ]:
#@title
def find_how_many_you_getV1_for_best_coef(x_i,alpha):
  actual_invarian = invV1(x_balance,y_balance,alpha)
  y_left = 0.000000001*y_balance
  y_right = 10*y_balance 
  for _ in range(100):
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV1(x_i,y_left,alpha)
    invariant_delta_in_right = actual_invarian - invV1(x_i,y_right,alpha)
    invariant_delta_in_mean = actual_invarian - invV1(x_i,mean,alpha)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance - mean  
  return y_balance - mean  



def find_how_many_you_getV2_for_best_coef(x_balance,y_balance,x_balance_new,alpha,betta):
  actual_invarian = invV2(x_balance,y_balance,alpha,betta)
  y_left = 1
  y_right = 100*y_balance 
  for _ in range(500):
    
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV2(x_balance_new,y_left,alpha,betta)
    invariant_delta_in_right = actual_invarian - invV2(x_balance_new,y_right,alpha,betta)
    invariant_delta_in_mean = actual_invarian - invV2(x_balance_new,mean,alpha,betta)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance - mean  
  return y_balance - mean

def has_negative(arr):
  for i in arr:
    if i<0:
      return True
  return False

x_flatV1 = []
y_flatV1 = []
x_flatV2 = []
y_flatV2 = []
x_line = []
y_line = []

for x_i in np.linspace(0.3*x_balance,2*x_balance,30):
    y_i = x_balance+y_balance-x_i
    x_line.append(x_i)
    y_line.append(y_i)

curve_sum = 1000000000000000000000000

alpha_for_V2_best = 0.05
for alpha_for_V1 in np.linspace(0.05,1,10):
  y_flatV1 = []
  for x_i in np.linspace(0.3*x_balance,2*x_balance,30):
      y_i = y_balance - find_how_many_you_getV1_for_best_coef(x_i,alpha_for_V1)
      y_flatV1.append(y_i)
  diff_in_values_V1_line = np.array(y_flatV1) - np.array(y_line)
  
  if has_negative(diff_in_values_V1_line):
    continue
 
  if diff_in_values_V1_line.sum() < curve_sum:
    curve_sum = diff_in_values_V1_line.sum()
    alpha_for_V1_best = alpha_for_V1

alpha_for_V2_best = 0.05
for alpha_for_V1 in np.linspace(0.05,1,10):
  y_flatV1 = []
  for x_i in np.linspace(0.3*x_balance,2*x_balance,30):
      y_i = y_balance - find_how_many_you_getV1_for_best_coef(x_i,alpha_for_V1)
      y_flatV1.append(y_i)
  diff_in_values_V1_line = np.array(y_flatV1) - np.array(y_line)
  
  if has_negative(diff_in_values_V1_line):
    continue
 
  if diff_in_values_V1_line.sum() < curve_sum:
    curve_sum = diff_in_values_V1_line.sum()
    alpha_for_V1_best = alpha_for_V1

curve_sum = 1000000000000000000000000

alpha_for_V2_best = 0.05
betta_for_V2_best = 0.05
alpha_for_V2_start = 0.05
betta_for_V2_start = 0.05
for alpha_for_V2 in np.linspace(alpha_for_V2_start,1,100):
  for betta_for_V2 in np.linspace(betta_for_V2_start,1,100):
    y_flatV2 = []
    for x_i in np.linspace(0.3*x_balance,2*x_balance,30):
        y_i = y_balance - find_how_many_you_getV2_for_best_coef(x_balance,y_balance,x_i,alpha_for_V2,betta_for_V2)
        y_flatV2.append(y_i)
    diff_in_values_V2_line = np.array(y_flatV2) - np.array(y_line)
    #print(alpha_for_V2,betta_for_V2,curve_sum,diff_in_values_V2_line.sum(),has_negative(diff_in_values_V2_line))
    if has_negative(diff_in_values_V2_line):
        continue
 
    #print()
    if diff_in_values_V2_line.sum() < curve_sum:
      curve_sum = diff_in_values_V2_line.sum()
      alpha_for_V2_best = alpha_for_V2
      betta_for_V2_best = betta_for_V2

print("alpha_V1:" + str(alpha_for_V1_best))
print("alpha_V2:" + str(alpha_for_V2_best) +", bettta_V2:" + str(betta_for_V2_best))

NameError: ignored


---


### Графики: cравнение uniswap, flatV1  и flatV2

В v1 один параметр (*alpha*), в v2 два 


$skewness = 0.5(\frac{x}{y}+\frac{y}{x}) $

**v1**: $inv = (x+y) \cdot skewness^{-\alpha}+ (x \cdot y)\cdot skewness^{\alpha} $

**v2**: $inv = (x+y) \cdot skewness^{-\alpha}+ 2\cdot(x \cdot y)^{0.5}\cdot (skewness-\beta)^{\alpha} $

*alpha* - влияние скошенности портфеля. Чем больше *alpha*, тем быстрее убывает вклад инварианта суммы x+y и возрастает вклад инварианта y*x

*betta* - "прижатость" к прямолинейной зависимости




(X_old + Y_old)*sk + (X_old * Y_old)*sk = 1234
(X_new + Y_new)*sk + (X_new * Y_new)*sk = 1234

In [ ]:
x_balance = 500000000000 #100m with 8 digits
y_balance = x_balance
alpha_for_V1 = 0.47
alpha_for_V2 = 0.50
betta_for_V2 = 0.46


In [ ]:
#@title
x_flatV1 = []
y_flatV1 = []
x_flatV2 = []
y_flatV2 = []
x_line = []
y_line = []
x_uniswap = []
y_uniswap = []

# for x_i in np.linspace(0.1*x_balance,1.9*x_balance,30):
#     y_i = y_balance - find_how_many_you_getV1(x_i,alpha_for_V1)
#     x_flatV1.append(x_i)
#     y_flatV1.append(y_i)
for x_i in np.linspace(0.1*x_balance,1.9*x_balance,30):
    y_i = y_balance - find_how_many_you_getV2(x_balance,y_balance,x_i,alpha_for_V2,betta_for_V2)
    x_flatV2.append(x_i)
    y_flatV2.append(y_i)

for x_i in np.linspace(0.1*x_balance,1.9*x_balance,30):
    y_i = x_balance+y_balance-x_i
    x_line.append(x_i)
    y_line.append(y_i)

for x_i in np.linspace(0.1*x_balance,1.9*x_balance,30):
    y_i = x_balance*y_balance/x_i
    x_uniswap.append(x_i)
    y_uniswap.append(y_i)

# diff_in_values_V1_line = np.array(y_flatV1) - np.array(y_line)
diff_in_values_V2_line = np.array(y_flatV2) - np.array(y_line)
# for i in diff_in_values_V1_line:
#   if i<0:
#     print("!ERROR IN V1: V1 curve under line!") 
#     break

for i in diff_in_values_V2_line:
  if i<0:
    print("!ERROR IN V2: V2 curve under line!")
    break

fig = go.Figure()

# fig.add_trace(go.Scatter(x=x_flatV1, y=y_flatV1,
#                     mode ='lines', name = 'flatV1, alpha =' + str(alpha_for_V1)))

fig.add_trace(go.Scatter(x=x_flatV2, y=y_flatV2,
                    mode ='lines', name = 'flatV2, alpha =' + str(alpha_for_V2) + ", beta =" +str(betta_for_V2)))
fig.add_trace(go.Scatter(x=x_uniswap, y=y_uniswap,
                         
                    mode = 'lines', name = 'uniswap'))

fig.add_trace(go.Scatter(x=x_line, y=y_line,
                    mode = 'lines', name = 'line'))

fig.update_xaxes(range=[0,2*x_balance])
fig.update_yaxes(range=[0,2*x_balance])

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    plot_bgcolor='rgb(224,224,224)',
    paper_bgcolor = 'rgb(224,224,224)',
    title="Bonding curves",
    xaxis_title = "x amount",
    yaxis_title = "y amount")




---




### Калькулятор: cравнение uniswap, flatV1  и flatV2
В v1 один параметр (*alpha*), в v2 два 


$skewness = 0.5(\frac{x}{y}+\frac{y}{x}) $

**v1**: $inv = (x+y) \cdot skewness^{-\alpha}+ (x \cdot y)\cdot skewness^{\alpha} $

**v2**: $inv = (x+y) \cdot skewness^{-\alpha}+ 2\cdot(x \cdot y)^{0.5}\cdot (skewness-\beta)^{\alpha} $

*alpha* - влияние скошенности портфеля. Чем больше *alpha*, тем быстрее убывает вклад инварианта суммы x+y и возрастает вклад инварианта y*x

*betta* - "прижатость" к прямолинейной зависимости

In [ ]:
x_balance = 2000009999 #500k with 6 digits
y_balance = 2000000000
alpha_for_V2 = 0.50
betta_for_V2 = 0.46
#you_pay_in_x = 10000000000
you_pay_in_x = 1000000
               


In [ ]:
#@title

updated_X_amount = x_balance + you_pay_in_x
you_get_flat_v2 = find_how_many_you_getV2(x_balance,y_balance,updated_X_amount,alpha_for_V2,betta_for_V2)
you_get_uniswap = find_how_many_you_get_uniswap(updated_X_amount)

price_in_contract =  x_balance/y_balance
#price_flat_v1 = you_pay_in_x/you_get_flat_v1
price_flat_v2 = you_pay_in_x/you_get_flat_v2
price_uniswap = you_pay_in_x/you_get_uniswap

#slippage_flat_v1 = price_in_contract/price_flat_v1
slippage_flat_v2 = price_in_contract/price_flat_v2
slippage_uniswap = price_in_contract/price_uniswap

display_table([["","balance x","balance y"],
              ["",x_balance,y_balance]])
print()
data = [["","you pay in x","you get in y(without fee)","you get in y(with fee 0.05%)","governance fee(0.02/0.05 of fee)","buy price","slippage % (price in contract x/y to actual price)"],
        ["flat v2",you_pay_in_x,you_get_flat_v2,(1-0.0005)*you_get_flat_v2,(0.0002)*you_get_flat_v2,price_flat_v2,100*(1-slippage_flat_v2)],
        ["uniswap",you_pay_in_x,you_get_uniswap,(1-0.0005)*you_get_uniswap,(0.0002)*you_get_uniswap,price_uniswap,100*(1-slippage_uniswap)]]
display_table(data)

print()

display_table([["","balance x  new","balance y new"],
              ["",x_balance+you_pay_in_x, y_balance - you_get_flat_v2+(0.0003)*you_get_flat_v2]])




---




### Калькулятор: cравнение uniswap и flatV2, расчёты аналогичные RIDE

Отличие этого блока от предыдущего в том, что инвариант расчитывается аналогично Ride


В v1 один параметр (*alpha*), в v2 два 


$skewness = 0.5(\frac{x}{y}+\frac{y}{x}) $

**v1**: $inv = (x+y) \cdot skewness^{-\alpha}+ (x \cdot y)\cdot skewness^{\alpha} $

**v2**: $inv = (x+y) \cdot skewness^{-\alpha}+ 2\cdot(x \cdot y)^{0.5}\cdot (skewness-\beta)^{\alpha} $

*alpha* - влияние скошенности портфеля. Чем больше *alpha*, тем быстрее убывает вклад инварианта суммы x+y и возрастает вклад инварианта y*x

*betta* - "прижатость" к прямолинейной зависимости


In [ ]:
x_balance = 2028062629151
y_balance = 2731131039730
alpha_for_V2 = 0.50
betta_for_V2 = 0.46
you_pay_in_x = 100000000

def find_how_many_you_getV2(x_balance,y_balance,x_balance_new,alpha,betta):
  actual_invarian = invV2(x_balance,y_balance,alpha,betta)
  y_left = 1
  y_right = 100*y_balance 
  for _ in range(50):
    mean = (y_left + y_right)/2
    invariant_delta_in_left = actual_invarian - invV2(x_balance_new,y_left,alpha,betta)
    invariant_delta_in_right = actual_invarian - invV2(x_balance_new,y_right,alpha,betta)
    invariant_delta_in_mean = actual_invarian - invV2(x_balance_new,mean,alpha,betta)
    if invariant_delta_in_mean*invariant_delta_in_left < 0:
      y_left = y_left
      y_right = mean
    elif invariant_delta_in_mean*invariant_delta_in_right <0:
      y_left = mean
      y_right = y_right
    else:
      return y_balance -  mean
  return y_balance -  mean

#print (find_how_many_you_getV2(x_balance, y_balance, x_balance+you_pay_in_x, alpha_for_V2, betta_for_V2))

In [ ]:
#@title
# rewrite as Ride logic

updated_X_amount = x_balance + you_pay_in_x
you_get_flat_v2 = int(find_how_many_you_getV2(x_balance,y_balance,updated_X_amount,alpha_for_V2,betta_for_V2))
you_get_uniswap = find_how_many_you_get_uniswap(updated_X_amount)

price_in_contract =  x_balance/y_balance
#price_flat_v1 = you_pay_in_x/you_get_flat_v1
price_flat_v2 = you_pay_in_x/you_get_flat_v2
price_uniswap = you_pay_in_x/you_get_uniswap

#slippage_flat_v1 = price_in_contract/price_flat_v1
slippage_flat_v2 = price_in_contract/price_flat_v2
slippage_uniswap = price_in_contract/price_uniswap

display_table([["","balance x","balance y"],
              ["",x_balance,y_balance]])
print()
data = [["","you pay in x","you get in y(without fee)","you get in y(with fee 0.05%)","governance fee(0.02/0.05 of fee)","buy price","slippage % (price in contract x/y to actual price)"],
        ["flat v2",you_pay_in_x,you_get_flat_v2,(1-0.0005)*you_get_flat_v2,(0.0002)*you_get_flat_v2,price_flat_v2,100*(1-slippage_flat_v2)],
        ["uniswap",you_pay_in_x,you_get_uniswap,(1-0.0005)*you_get_uniswap,(0.0002)*you_get_uniswap,price_uniswap,100*(1-slippage_uniswap)]]
display_table(data)

print()

display_table([["","balance x  new","balance y new(without fee)"],
              ["",x_balance+you_pay_in_x, y_balance - you_get_flat_v2 ]])



---



### Бонусы и штрафы при внесении только одного токена: Uniswap
В начальный мометн времени обменники находятся в состоянии равновесия

Происходит смещение портфеля так, что новое значение токенов х равно **x_balance_after_some_time**

Второй депозитор вносит **deposit_in_x** токенов x

В результате расчётов показывается, какое количество токенов может забрать первый и второй депозитор в зависимости от цены ассетов

In [ ]:
x_balance_start = 1000 
y_balance_start = x_balance_start

x_balance_after_some_time = 0.9*x_balance_start #before new depositor come
deposit_in_x = 100

In [ ]:
#@title
invariant_start = x_balance_start*y_balance_start
share_token_start = (x_balance_start*y_balance_start)**(0.5)

#logic from https://docs.google.com/spreadsheets/d/1KO6gneFBoKCL7vBG3d6r06qGf6aMxRvA8hZcC2W5yBQ/edit#gid=1946128407
y_balance_after_some_time = invariant_start/x_balance_after_some_time
price_after_disbalance = x_balance_after_some_time/y_balance_after_some_time

x_balance_after_deposit = x_balance_after_some_time + deposit_in_x
y_balance_after_deposit = y_balance_after_some_time
price_after_diposit = x_balance_after_deposit/y_balance_after_deposit
invariant_after_deposit = x_balance_after_deposit*y_balance_after_deposit

y_balance_first_old_invariant_new_price = (invariant_start/price_after_diposit)**0.5
share_tokens_second_depositors = y_balance_after_deposit/y_balance_first_old_invariant_new_price*share_token_start - share_token_start 

balance_x_first_after_deposit = x_balance_after_deposit*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_x_second_after_deposit = x_balance_after_deposit*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)
balance_y_first_after_deposit = y_balance_after_deposit*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_y_second_after_deposit = y_balance_after_deposit*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)


balance_x_after_deposit_in_equilibrium = invariant_after_deposit**0.5
balance_y_after_deposit_in_equilibrium = invariant_after_deposit**0.5
price_in_equlibrium = balance_x_after_deposit_in_equilibrium/balance_y_after_deposit_in_equilibrium

balance_x_first_after_deposit_equal = balance_x_after_deposit_in_equilibrium*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_x_second_after_deposit_equal = balance_x_after_deposit_in_equilibrium*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)
balance_y_first_after_deposit_equal = balance_y_after_deposit_in_equilibrium*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_y_second_after_deposit_equal = balance_y_after_deposit_in_equilibrium*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)

balance_y_after_deposit_in_disbalance  = (invariant_after_deposit/price_after_disbalance)**0.5
balance_x_after_deposit_in_disbalance = invariant_after_deposit/balance_y_after_deposit_in_disbalance

balance_x_first_after_deposit_disbalance = balance_x_after_deposit_in_disbalance*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_x_second_after_deposit_disbalance = balance_x_after_deposit_in_disbalance*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)
balance_y_first_after_deposit_disbalance = balance_y_after_deposit_in_disbalance*share_token_start/(share_token_start+share_tokens_second_depositors)
balance_y_second_after_deposit_disbalance = balance_y_after_deposit_in_disbalance*share_tokens_second_depositors/(share_token_start+share_tokens_second_depositors)

print("price in equlibrium: " +str(price_in_equlibrium))
print("price in contract after disbalance: " + str(price_after_disbalance))
print("price in contract after deposit: " + str(price_after_diposit))
print()



print("withdraw after deposit with price " + str(price_in_equlibrium))
data = [["","x","y"],
        ["first depositor",balance_x_first_after_deposit_equal,balance_y_first_after_deposit_equal],
        ["second depositor",balance_x_second_after_deposit_equal,balance_y_second_after_deposit_equal],
        ]
display_table(data)
print()
print()
print()

print("withdraw after deposit with price " + str(price_after_diposit))
data = [["","x","y"],
        ["first depositor",balance_x_first_after_deposit,balance_y_first_after_deposit],
        ["second depositor",balance_x_second_after_deposit,balance_y_second_after_deposit],
        ]
display_table(data)

print()
print()
print()

print("withdraw after deposit with price " + str(price_after_disbalance))
data = [["","x","y"],
        ["first depositor",balance_x_first_after_deposit_disbalance,balance_y_first_after_deposit_disbalance],
        ["second depositor",balance_x_second_after_deposit_disbalance,balance_y_second_after_deposit_disbalance],
        ]
display_table(data)





---

###  Пополнение одним токеном: flatV2



В начальный мометн времени обменники находятся в состоянии равновесия


Второй депозитор вносит **deposit_in_x** токенов x

В результате расчётов выводтся аргументы функции - количество сколько пользователь должен заплатить и сколько получить в ходе "виртуального" свопа


In [ ]:

x_balance = 161655198219 #159721291500 #500k with 6 digits
y_balance = 206094774232 #211452045402
s_supply = 1000000000
managment_staking_fee = 0.27*1e6 
alpha = 0.50
betta = 0.46
deposit_in_x = 398103000  #0.1*x_balance_start


In [ ]:
#@title


def find_same_proportion(x_balance,y_balance,deposit_in_x):  
  x_left = 1
  x_right = deposit_in_x
  for _ in range(50):
    x_virtual_swap_pay = int((x_left + x_right)/2)
    y_virtual_swap_get = int(find_how_many_you_getV2(x_balance,
                        y_balance,
                        x_balance+x_virtual_swap_pay,
                        alpha,
                        betta))
    
    ratio_in_contract_aftrer_virtual_swap = (x_balance+x_virtual_swap_pay)/(y_balance-y_virtual_swap_get)
    
    x_to_deposit = deposit_in_x - x_virtual_swap_pay 
    y_to_deposit = y_virtual_swap_get 
    # print(x_left,x_right)
    # print(ratio_in_contract_aftrer_virtual_swap,x_to_deposit/y_to_deposit, ratio_in_contract_aftrer_virtual_swap/(x_to_deposit/y_to_deposit))
    # print()

    ratio_virtual_replanish = int(x_to_deposit)/int(y_to_deposit)
    print(ratio_in_contract_aftrer_virtual_swap-ratio_virtual_replanish)
    if ratio_in_contract_aftrer_virtual_swap-ratio_virtual_replanish > 0:
      x_left = x_left
      x_right = x_virtual_swap_pay
    elif ratio_in_contract_aftrer_virtual_swap-ratio_virtual_replanish < 0 :
      x_left = x_virtual_swap_pay
      x_right = x_right
    else:
      print("first and second arg in replanishmentWithOneToken")
      print("x_virtual_swap_pay",x_virtual_swap_pay,", y_virtual_swap_get",y_virtual_swap_get)
      print()
      print("your share after swap: ",y_virtual_swap_get/(y_balance-y_virtual_swap_get))
      return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get
  print("first and second arg in replanishmentWithOneToken")
  print("x_virtual_swap_pay",int(x_virtual_swap_pay))
  print("y_virtual_swap_get",int(y_virtual_swap_get))
  print()

  ###
  X_ = x_balance+x_virtual_swap_pay
  Y_ = y_balance-y_virtual_swap_get
  sh_x = (x_to_deposit-managment_staking_fee)/X_
  sh_y = (y_to_deposit)/Y_
  print(sh_x,sh_y)
  print(sh_x*y_balance/1e6,sh_y*y_balance/1e6)
  print(sh_x*(x_balance+deposit_in_x)/1e6,sh_y*(x_balance+deposit_in_x)/1e6)
  # print(s_supply*(x_to_deposit-swap_comission)/X_,s_supply*y_to_deposit/Y_,(x_to_deposit/X_)/(y_to_deposit/Y_))
  # print("get correct",sh_max*(x_balance+deposit_in_x))
  # print("get less",sh_min*(x_balance+deposit_in_x) )
  # print("get y",sh_y*(x_balance+deposit_in_x),)
  # print(sh_max*(x_balance+deposit_in_x),sh_min*(x_balance+deposit_in_x),(sh_min*(x_balance+deposit_in_x))/(sh_max*(x_balance+deposit_in_x)))
  # print(sh_min*(x_balance+deposit_in_x)/1e6,sh_max*(x_balance+deposit_in_x)/1e6,(sh_min*(y_balance))/(sh_max*(y_balance)))
  # print(sh_min*(y_balance)/1e6,sh_max*(y_balance)/1e6)
  # print(sh_min*s_supply)
  ###
  
  return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get

invariant = invV2(x_balance,y_balance,alpha,betta)
share_token = (x_balance*y_balance)**(0.5)

x_to_replanish,y_to_replanish = find_same_proportion(x_balance,y_balance,deposit_in_x)


---

###  Пополнение одним токеном c комиссией: flatV2

Отличие от предыдущего блока в том, что внутренний своп содержит комиссию

Мотивация к введению: сейчас через операцию пополнения в одном токене/снятия ликвидонсти можно получить другой токен выгоднее, нежели через обмен, т.к. не участвуют комиссии

Логика решения

![](https://drive.google.com/uc?export=view&id=1iBAo1GDJ_9HVcqtd-Bw_hrp8PVHUrW1r)

In [ ]:
x_balance = 170353426200 #500k with 6 digits
y_balance = 203043614144
alpha = 0.50
betta = 0.46
comission = 500
comissionDelimiter = 10000000
deposit_in_x = 1000000 #0.1*x_balance_start

In [ ]:
#@title

def find_same_proportion(x_balance,y_balance,deposit_in_x):
  x_left = 1
  x_right = deposit_in_x
  for _ in range(50):
    x_virtual_swap_pay_before_comission = (x_left + x_right)/2
    x_virtual_swap_pay = x_virtual_swap_pay_before_comission
    y_virtual_swap_get_without_comission = find_how_many_you_getV2(x_balance,
                        y_balance,
                        x_balance+x_virtual_swap_pay,
                        alpha,
                        betta)
    y_virtual_swap_get = y_virtual_swap_get_without_comission - y_virtual_swap_get_without_comission*comission/comissionDelimiter
    ratio_in_contract_aftrer_virtual_swap = (x_balance+x_virtual_swap_pay)/(y_balance-y_virtual_swap_get)
    x_to_deposit = deposit_in_x - x_virtual_swap_pay
    y_to_deposit = y_virtual_swap_get
    ratio_virtual_replanish = x_to_deposit/y_to_deposit
    inv_new = invV2(x_balance+x_virtual_swap_pay,y_balance-y_to_deposit,alpha,betta)
    inv_old = invV2(x_balance,y_balance,alpha,betta)
    if ratio_virtual_replanish-ratio_in_contract_aftrer_virtual_swap < 0:
      x_left = x_left
      x_right = x_virtual_swap_pay
    elif ratio_virtual_replanish - ratio_in_contract_aftrer_virtual_swap > 0 :
      x_left = x_virtual_swap_pay
      x_right = x_right
    else:
      print("first and second arg in replanishmentWithOneToken")
      print("x_virtual_swap_pay",x_virtual_swap_pay,", y_virtual_swap_get",y_virtual_swap_get)
      print('you pay in x', deposit_in_x)
      print("comission in x", deposit_in_x*comission/comissionDelimiter)
      print("balance x after swap ", x_balance+x_virtual_swap_pay,"balance y after swap ", y_balance - y_virtual_swap_get_without_comission)
      print("your_share: ", x_to_deposit/(x_balance+x_virtual_swap_pay),y_to_deposit/(y_balance - y_virtual_swap_get))
      print("invariants skeweness: ", invV2(x_balance,y_balance,alpha,betta)/invV2(x_balance+x_virtual_swap_pay,y_balance - y_virtual_swap_get,alpha,betta) )
      return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get
  
  print("first and second arg in replanishmentWithOneToken")
  print("x_virtual_swap_pay",x_virtual_swap_pay,", y_virtual_swap_get",y_virtual_swap_get)
  print('you pay in x', deposit_in_x)
  print("comission in y", deposit_in_x*comission/comissionDelimiter)
  print("balance x after swap ", x_balance+x_virtual_swap_pay,"balance y after swap ", y_balance - y_virtual_swap_get_without_comission)
  print("your_share A token and B token: ", x_to_deposit/(x_balance+x_virtual_swap_pay),y_to_deposit/(y_balance - y_virtual_swap_get))
  print("invariants skeweness: ", invV2(x_balance,y_balance,alpha,betta)/invV2(x_balance+x_virtual_swap_pay,y_balance - y_virtual_swap_get_without_comission,alpha,betta) )
  print(y_balance,y_virtual_swap_get_without_comission)
  return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get

invariant = invV2(x_balance,y_balance,alpha,betta)
share_token = (x_balance*y_balance)**(0.5)

x_to_replanish,y_to_replanish = find_same_proportion(x_balance,y_balance,deposit_in_x)

first and second arg in replanishmentWithOneToken
x_virtual_swap_pay 544207.0042572024 , y_virtual_swap_get 543255.111032727
you pay in x 1000000
comission in y 50.0
balance x after swap  170353970407.00427 balance y after swap  203043070861.72485
your_share A token and B token:  2.675564265709989e-06 2.675565872080471e-06
invariants skeweness:  1.0
203043614144 543282.2751464844


In [ ]:
#@title

def invV2(x_i,y_i,alpha,betta):
  skeweness = 1/2*(y_i/x_i+x_i/y_i)
  return (x_i+y_i)*(skeweness)**(-alpha) + 2*(x_i*y_i)**(1/2)*(skeweness - betta)**((alpha))

def find_same_proportion(x_balance,y_balance,deposit_in_x):
  
  x_left = 1
  x_right = deposit_in_x
  for _ in range(50):
    x_virtual_swap_pay = (x_left + x_right)/2
    y_virtual_swap_get = find_how_many_you_getV2(x_balance,
                        y_balance,
                        x_balance+x_virtual_swap_pay,
                        alpha,
                        betta)
    
    ratio_in_contract_aftrer_virtual_swap = (x_balance+x_virtual_swap_pay)/(y_balance-y_virtual_swap_get)
    x_to_deposit = deposit_in_x - x_virtual_swap_pay
    y_to_deposit = y_virtual_swap_get
    ratio_virtual_replanish = int(x_to_deposit)/int(y_to_deposit)

    if ratio_in_contract_aftrer_virtual_swap-ratio_virtual_replanish > 0:
      x_left = x_left
      x_right = x_virtual_swap_pay
    elif ratio_in_contract_aftrer_virtual_swap-ratio_virtual_replanish < 0 :
      x_left = x_virtual_swap_pay
      x_right = x_right
    else:
      return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get
  return deposit_in_x-x_virtual_swap_pay,y_virtual_swap_get
x= []
y= []
for x_depos in np.arange(5000000,10000000,1000):
  a1,a2 = find_same_proportion(x_balance,y_balance,x_depos)
  x.append(int(x_depos))
  y.append(a1/a2)

fig = go.Figure()

# fig.add_trace(go.Scatter(x=x_flatV1, y=y_flatV1,
#                     mode ='lines', name = 'flatV1, alpha =' + str(alpha_for_V1)))

fig.add_trace(go.Scatter(x=x, y=y,
                    mode ='lines', name = 'flatV2'))
